<a href="https://colab.research.google.com/github/samyuktaprabhu/DMPA_LAB/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [50]:
from google.colab import files
uploaded = files.upload()

Saving wikipedia_dataset_output.csv to wikipedia_dataset_output (1).csv


In [0]:
df=pd.read_csv('wikipedia_dataset_output.csv')

In [52]:
df

,Index,Wiki_output
0,0,Wikipedia ( (listen) wik-ih-PEE-dee-É™ or (li...
1,1,The Audi A4 is a line of compact executive car...
2,2,Wikipedia ( (listen) wik-ih-PEE-dee-É™ or (li...
3,3,Vivanta is an Indian hotel chain established i...
4,4,Wikipedia ( (listen) wik-ih-PEE-dee-É™ or (li...
...,...,...
494,494,Tractors in India is a major industry and sign...
495,495,"Mumbai (English: , Marathi: [ËˆmumbÉ™i]; also ..."
496,496,Bajaj Auto Limited is an Indian global two-whe...
497,497,Bajaj Electricals Ltd (BEL) (BSE: 500031) is a...


In [0]:
wordd=df['Wiki_output'][0]

In [0]:
df = df[df.Wiki_output != wordd]

In [0]:
df.to_csv('hello.csv')

In [0]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [35]:
import nltk
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [36]:
#nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [91]:
import random
text_data = []
with open('hello.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .5:
            print(tokens)
            text_data.append(tokens)

['1,1,"the', 'compact', 'executive', 'produce', 'since', 'german', 'manufacturer', 'subsidiary', 'volkswagen', 'group']
['6,6,"a', 'privately', 'company', 'private', 'company', 'close', 'corporation', 'business', 'company', 'own', 'either', 'governmental', 'organization', 'relatively', 'small', 'number', 'shareholder', 'company', 'member', 'offer', 'trade', 'company', 'stock', 'share', 'general', 'public', 'stock', 'market', 'exchange', 'rather', 'company', 'stock', 'offer', 'own', 'trade', 'exchange', 'privately', 'counter', 'ambiguous', 'terms', 'privately', 'company', 'closely', 'corporation', 'unquoted', 'company', 'unlisted', 'company']
['though', 'visible', 'publicly', 'trade', 'counterpart', 'private', 'company', 'major', 'importance', 'world', 'economy', 'large', 'private', 'company', 'unite', 'state', 'account', '1,800,000,000,000', 'trillion', 'revenue', 'employ', 'million', 'people', 'accord', 'forbes', 'using', 'substantially', 'smaller', 'comparison', 'company', 'forbes', 

In [92]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [93]:
import gensim
NUM_TOPICS =7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.026*"exchange" + 0.023*"stock" + 0.021*"india" + 0.019*"company"')
(1, '0.041*"company" + 0.029*"india" + 0.019*"limited" + 0.014*"indian"')
(2, '0.009*"company" + 0.008*"award" + 0.006*"storage" + 0.006*"series"')
(3, '0.023*"company" + 0.022*"group" + 0.017*"india" + 0.012*"indian"')
(4, '0.035*"company" + 0.019*"india" + 0.013*"group" + 0.012*"power"')
(5, '0.012*"world" + 0.009*"first" + 0.008*"vehicle" + 0.008*"manufacturer"')
(6, '0.017*"india" + 0.015*"large" + 0.014*"adani" + 0.013*"limited"')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [94]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [95]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)


/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [96]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.009*"company" + 0.008*"india" + 0.007*"indian" + 0.007*"world"')
(1, '0.021*"company" + 0.017*"india" + 0.012*"adani" + 0.011*"group"')
(2, '0.034*"company" + 0.025*"india" + 0.016*"indian" + 0.015*"group"')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [97]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
